In [1]:
import os
import shutil
import stat
import seaborn as sns
import collections
import h5py
import numpy as np
import tensorflow as tf
import matplotlib.image as img
import random
import cv2
import PIL
import matplotlib.pyplot as plt
import matplotlib.image as img
from os import listdir
from os.path import isfile, join
from collections import defaultdict
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from sklearn.model_selection import train_test_split
from skimage.io import imread
from keras.utils.np_utils import to_categorical
from keras.applications.inception_v3 import preprocess_input
from keras.models import load_model
from shutil import copy
from shutil import copytree, rmtree
import tensorflow.keras.backend 
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras import regularizers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow import keras
from tensorflow.keras import models

In [4]:
import os

def check_image_files(train_dir, test_dir):
    train_images = set([file for file in os.listdir(train_dir) if os.path.isfile(os.path.join(train_dir, file))])
    test_images = set([file for file in os.listdir(test_dir) if os.path.isfile(os.path.join(test_dir, file))])
    
    common_images = train_images.intersection(test_images)
    
    if common_images:
        print("Eğitim ve test dizinlerinde aynı isimde {} resim dosyası bulunmaktadır.".format(len(common_images)))
        print("Örnek ortak resim dosyaları:", list(common_images)[:5])
    else:
        print("Eğitim ve test dizinlerinde aynı isimde resim dosyası bulunmamaktadır.")

train_data = r"C:\Users\kurt_\Desktop\food-101\train"
test_data = r"C:\Users\kurt_\Desktop\food-101\test"

check_image_files(train_data, test_data)


Eğitim ve test dizinlerinde aynı isimde resim dosyası bulunmamaktadır.


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, LambdaCallback
from tensorflow.keras import regularizers

import tensorflow.keras.backend as K

# Clear Keras session
K.clear_session()

# Define constants
n_classes = 101
batch_size = 32
width, height = 331, 331
train_data = r"C:\Users\kurt_\Desktop\food-101\train"
test_data = r"C:\Users\kurt_\Desktop\food-101\test"
train_samples = 75750
test_samples = 25250

# Data generators
train_data_gen = ImageDataGenerator(rescale=1. / 255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_data_gen = ImageDataGenerator(rescale=1. / 255)
train_gen = train_data_gen.flow_from_directory(train_data, target_size=(height, width), batch_size=batch_size, class_mode='categorical')
test_gen = test_data_gen.flow_from_directory(test_data, target_size=(height, width), batch_size=batch_size, class_mode='categorical')

# Model
inception = InceptionV3(weights='imagenet', include_top=False)
layer = inception.output
layer = GlobalAveragePooling2D()(layer)
layer = Dense(128, activation='relu')(layer)
layer = Dropout(0.2)(layer)
predictions = Dense(n_classes, kernel_regularizer=regularizers.l2(0.005), activation='softmax')(layer)
model = Model(inputs=inception.input, outputs=predictions)

# Compile model
optimizer = SGD(momentum=0.9)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks
checkpointer = ModelCheckpoint(filepath='best_model_101class_inceptionv3den6.hdf5', save_best_only=True)
csv_logger = CSVLogger('history_101class_inceptionv3den6.log')

# Progress print callback
def print_progress(epoch, logs):
    print("Epoch {}, Loss: {:.4f}, Accuracy: {:.4f}, Val Loss: {:.4f}, Val Accuracy: {:.4f}".format(
        epoch+1, logs['loss'], logs['accuracy'], logs['val_loss'], logs['val_accuracy']))

progress_printer = LambdaCallback(on_epoch_end=print_progress)

# Train the model
history_101class_inceptionv3den6 = model.fit(train_gen, 
                             steps_per_epoch=train_samples // batch_size, 
                             validation_data=test_gen, 
                             validation_steps=test_samples // batch_size, 
                             epochs=30, 
                             callbacks=[csv_logger, checkpointer, progress_printer])

# Save the model
model.save('model_trained_101class_inceptionv3den6.hdf5')


Found 75750 images belonging to 101 classes.
Found 25250 images belonging to 101 classes.
Epoch 1/30
2367/2367 [==============================] - 1591s 664ms/step - loss: 2.4819 - accuracy: 0.4592 - val_loss: 3.0233 - val_accuracy: 0.3854
Epoch 2/30
2367/2367 [==============================] - 1504s 635ms/step - loss: 1.6979 - accuracy: 0.6229 - val_loss: 1.6406 - val_accuracy: 0.6364
Epoch 3/30
2367/2367 [==============================] - 1473s 622ms/step - loss: 1.4543 - accuracy: 0.6812 - val_loss: 2.0744 - val_accuracy: 0.5507
Epoch 4/30
2367/2367 [==============================] - 1485s 627ms/step - loss: 1.3033 - accuracy: 0.7173 - val_loss: 1.4707 - val_accuracy: 0.6822
Epoch 5/30
2367/2367 [==============================] - 1482s 626ms/step - loss: 1.1786 - accuracy: 0.7461 - val_loss: 1.5213 - val_accuracy: 0.6827
Epoch 6/30
2367/2367 [==============================] - 1436s 606ms/step - loss: 1.0791 - accuracy: 0.7688 - val_loss: 1.3991 - val_accuracy: 0.7083
Epoch 7/30
2367/

In [6]:
model_path = r"C:\Users\kurt_\best_model_101class_inceptionv3den6.hdf5"
test_data_path = r"C:\Users\kurt_\Desktop\food-101\test"

In [7]:
# Performans metriklerini hesaplamak için fonksiyon tanımla
def calculate_metrics(model, test_gen):
    # Modelden tahminler yap
    predictions = model.predict(test_gen)
    predicted_classes = np.argmax(predictions, axis=1)
    
    # Gerçek sınıfları al
    true_classes = test_gen.classes
    
    # Sınıf etiketlerini al
    class_labels = list(test_gen.class_indices.keys())

    # Doğruluk, hassasiyet, geri çağırma ve F1 puanını hesapla
    accuracy = accuracy_score(true_classes, predicted_classes)
    precision = precision_score(true_classes, predicted_classes, average='weighted')
    recall = recall_score(true_classes, predicted_classes, average='weighted')
    f1 = f1_score(true_classes, predicted_classes, average='weighted')

    return accuracy, precision, recall, f1

In [8]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# Boyutları tanımla
height, width = 331, 331

# Batch boyutunu tanımla
batch_size = 16

# Test veri üreteci oluştur
test_data_gen = ImageDataGenerator(rescale=1. / 255)
test_gen = test_data_gen.flow_from_directory(test_data_path, target_size=(height, width), batch_size=batch_size, class_mode='categorical', shuffle=False)

# Modeli yükle
model = load_model(model_path, compile=False)

# Performans metriklerini hesapla
accuracy, precision, recall, f1_score = calculate_metrics(model, test_gen)

# Hesaplanan metrikleri yazdır
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)


Found 25250 images belonging to 101 classes.
1579/1579 [==============================] - 99s 61ms/step
Accuracy: 0.7512475247524752
Precision: 0.7726273261431389
Recall: 0.7512475247524752
F1 Score: 0.7515048185504755
